In [1]:
from bastionlab import Connection, Identity, LogisticRegression
import polars as pl

In [48]:
data_owner = Identity.create("data_owner")

client = Connection("localhost", identity=data_owner).client

In [49]:
target_names = ["setosa", "versicolor", "virginica"]
df = pl.read_csv(
    "iris.csv",
    has_header=False,
    new_columns=[
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)",
        "target",
    ],
)


df = df.with_column(
    pl.col("target")
    .map(lambda a: a.apply(lambda b: True if target_names[b] == "virginica" else False))
    .alias("target_class")
)

target_class_df = pl.DataFrame({"classes": target_names})
target_class_df = target_class_df.with_row_count("id")

In [50]:
rdf = client.send_df(df)
classes_rdf = client.send_df(target_class_df)

train_rdf = rdf.select(pl.col(["petal length (cm)", "petal width (cm)"])).collect()
test_rdf = rdf.select(pl.col("target_class")).collect()
print(train_rdf, test_rdf)

FetchableLazyFrame(identifier=461f602f-ca5f-4e4c-8f2e-52f27f925c71) FetchableLazyFrame(identifier=8eacea3a-8076-44b0-a3fb-45b5f4c5cd3f)


In [51]:
trainer = client.train(train_rdf, test_rdf, LogisticRegression())

In [53]:
res = client.predict(trainer, [5.5, 4.2])
print(res)

FetchableLazyFrame(identifier=8cd288f3-040a-41fb-9f5a-c98817a555a9)
